In [3]:
# register model

import json
import mlflow
import logging
import os
import dagshub
from dotenv import load_dotenv
load_dotenv() 
# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)

# logging configuration
logger = logging.getLogger('model_registration')
logger.setLevel('DEBUG')

console_handler = logging.StreamHandler()
console_handler.setLevel('DEBUG')

file_handler = logging.FileHandler('model_registration_errors.log')
file_handler.setLevel('ERROR')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

def load_model_info(file_path: str) -> dict:
    """Load the model info from a JSON file."""
    try:
        with open(file_path, 'r') as file:
            model_info = json.load(file)
        logger.debug('Model info loaded from %s', file_path)
        return model_info
    except FileNotFoundError:
        logger.error('File not found: %s', file_path)
        raise
    except Exception as e:
        logger.error('Unexpected error occurred while loading the model info: %s', e)
        raise

def register_model(model_name: str, model_info: dict):
    """Register the model to the MLflow Model Registry."""
    try:
        model_uri = f"runs:/{model_info['run_id']}/{model_info['model_path']}"
        print(f"+++{model_uri}+++")
        # Register the model
        model_version = mlflow.register_model(model_uri, model_name)
        
        # Transition the model to "Staging" stage
        client = mlflow.tracking.MlflowClient()
        client.transition_model_version_stage(
            name=model_name,
            version=model_version.version,
            stage="Staging"
        )
        
        logger.debug(f'Model {model_name} version {model_version.version} registered and transitioned to Staging.')
    except Exception as e:
        logger.error('Error during model registration: %s', e)
        raise

def main():
    try:
        model_info_path = r'D:\yt-comment-sentiment-analysis2\experiment_info.json'
        model_info = load_model_info(model_info_path)
        
        model_name = "light-gbm-2.0"
        #register_model(model_name, model_info)
    except Exception as e:
        logger.error('Failed to complete the model registration process: %s', e)
        print(f"Error: {e}")

# if __name__ == '__main__':
#     main()

# Mlflow 3 MOdel Alisa and Tags

In [9]:
import json
import mlflow
import logging
import os
import dagshub
from mlflow import MlflowClient
from dotenv import load_dotenv
load_dotenv() 
# Set up DagsHub credentials for MLflow tracking
dagshub_token = os.getenv("DAGSHUB_PAT")
if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)


**registered Model Alias**

In [ ]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow")

# Assign the 'staging' alias to version 1
client.set_registered_model_alias(
    name="ligbm_model_v1", 
    alias="staging", 
    version="1"
)

**Load the model**

In [11]:
import mlflow
mlflow.set_tracking_uri(
    "https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow"
)
model = mlflow.lightgbm.load_model("models:/ligbm_model_v1@production")

In [4]:
import mlflow 
from mlflow import MlflowClient
client = MlflowClient(tracking_uri="https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow")
version_details = client.get_model_version_by_alias("ligbm_model_v1", "staging")
latest_version = version_details.version
latest_version

'1'

In [5]:
version_details

<ModelVersion: aliases=['staging'], creation_timestamp=1769664639968, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1769664639968, metrics=None, model_id=None, name='ligbm_model_v1', params=None, run_id='17ca7919ef6447638da9d341528d27dd', run_link='', source='models:/m-c047010f21cf483baf1a76660ef7b008', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [2]:
model

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,20
,learning_rate,0.09
,n_estimators,367
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


**registering new logged model and setting alias**

In [4]:
model_info_path = r'D:\yt-comment-sentiment-analysis2\experiment_info.json'
model_info = load_model_info(model_info_path)
model_info

2026-02-06 10:06:40,830 - model_registration - DEBUG - Model info loaded from D:\yt-comment-sentiment-analysis2\experiment_info.json


{'run_id': '37f8e073f230475ea3707094ff4acfb0', 'model_path': 'lgbm_model'}

In [5]:
from mlflow import MlflowClient
client = MlflowClient(tracking_uri="https://dagshub.com/Pranay5519/yt-comment-sentiment-analysis-2.mlflow")
model_name = "ligbm_model_v1"
model_path = model_info["model_path"]
run_id = model_info["run_id"]
model_uri = f"runs:/{run_id}/{model_path}"
model_uri

'runs:/37f8e073f230475ea3707094ff4acfb0/lgbm_model'

In [6]:
try:
    client.create_registered_model(model_name)
except Exception as e:
    print(e)
    #pass # Already exists

RESOURCE_ALREADY_EXISTS: Response: {'error_code': 'RESOURCE_ALREADY_EXISTS'}


In [ ]:
mv = client.create_model_version(
    name=model_name,
    source=model_uri,
    run_id=run_id
)

2026/02/06 10:10:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ligbm_model_v1, version 5


In [8]:
# 3. Push to "Staging" by assigning an Alias (The MLflow 3+ Standard)
client.set_registered_model_alias(
    name=model_name,
    alias="staging",
    version=mv.version
)
print(f"Model version {mv.version} is now registered and aliased to @staging")

Model version 5 is now registered and aliased to @staging
